In [53]:
import pandas as pd
import numpy as np
import datetime as dt


In [54]:
fault_codes = pd.read_excel('data/Service Fault Codes_1_0_0_167.xlsx')

D:\Anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [136]:
real_codes = pd.read_csv('data/J1939Faults.csv', low_memory = False)
onboard = pd.read_csv('data/VehicleDiagnosticOnboardData.csv', low_memory = False)

In [56]:
#real_codes = onboard.merge(j1939, how = 'left', left_on = 'FaultId', right_on = 'RecordID')

In [57]:
#real_codes = real_codes[real_codes['EquipmentID'].astype('str').str.len() < 6]

real_codes = real_codes[real_codes['EquipmentID'].str.len() < 6]

In [58]:
real_codes = real_codes[real_codes['ecuMake'] == 'CMMNS']

In [59]:
real_codes = real_codes.drop(columns=['actionDescription', 'faultValue', 'MCTNumber'])

In [60]:
real_codes = real_codes[real_codes['active'] == True]

In [61]:
#Selects lat and lon coordinates that are not in the bounded boxes

geo_codes = real_codes[(~real_codes['Latitude'].between(36.05942, 36.07392)) | (~real_codes['Longitude'].between(-86.44366, -86.42579))]
geo_codes = geo_codes[(~geo_codes['Latitude'].between(35.58108, 35.59558)) | (~geo_codes['Longitude'].between(-86.4528, -86.435))]
geo_codes = geo_codes[(~geo_codes['Latitude'].between(36.18775, 36.20225)) | (~geo_codes['Longitude'].between(-83.1837, -83.1658))]

#Here is an alternative method for selecting lats and lons that are not in the bounded boxes: 

#geo_codes = real_codes.query('(Latitude <= 36.05942 or Latitude >= 36.07392) or (Longitude <= -86.5419 or Longitude >= -86.3275)')


In [62]:
geo_codes = geo_codes.sort_values(["EquipmentID", "EventTimeStamp"], ascending = (False, True))

In [63]:
#Number of unique trucks in the data

geo_codes.EquipmentID.nunique()

928

In [64]:
#geo_codes.spnfmi.nunique()

In [65]:
geo_codes_select1569 = geo_codes[geo_codes['spn'].isin([1569])]

In [66]:
geo_codes_select5246 = geo_codes[geo_codes['spn'].isin([5246])]

In [69]:
#Number of unique 1569s thrown by the ECUs

geo_codes_select1569['RecordID'].nunique()

4277

In [70]:
#Number of unique 5426s thrown by the ECUs

geo_codes_select5246['RecordID'].nunique()

270

In [71]:
geo_codes.RecordID.nunique(())

200154

In [72]:
#Looking at the names of the metrics that are recorded when the ECU throws a code

geo_codes[geo_codes['RecordID']  == 4246].head(25)

,RecordID,ESS_Id,EventTimeStamp,eventDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,ecuSource,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,LocationTimeStamp
4245,4246,1048627,2015-02-24 13:45:06.000,Low (Severity Low) Catalyst Tank Level,05317106*04119044*051914190353*09400015*G1*BDR*,79751302,6X1u13D1500000000,CMMNS,0,1761,17,True,43,R1762,41.254166,-85.088888,2015-02-24 13:45:41.000


In [73]:
geo_codes.eventDescription.value_counts()

Low (Severity Low) Engine Coolant Level                               137206
Low (Severity Medium) Engine Coolant Level                             14676
High (Severity Low) Water In Fuel Indicator                             6992
Condition Exists Engine Protection Torque Derate                        4277
High Voltage (Water In Fuel Indicator)                                  3768
                                                                       ...  
Low Voltage (Common AC Auxiliaries Breaker)                                1
Abnormal Update Rate Second Fuel Level (Right Side)                        1
Out of Calibration Catalyst Tank Controller                                1
Not Reporting Data Aftertreatment Diesel Particulate Filter System         1
Error in System Catalyst Tank Controller                                   1
Name: eventDescription, Length: 266, dtype: int64

In [75]:
geo_codes = geo_codes.drop(columns=['LocationTimeStamp', 'active', 'ecuSource', 'ecuMake', 'ESS_Id'])

In [100]:
#Reorders our columns in a logical and readable way: index types, data types, others

geo_codes = geo_codes.reindex(columns = ['RecordID', 'EquipmentID', 'EventTimeStamp', 'spn', 'fmi', 'spnfmi',
                                         'activeTransitionCount', 'ecuModel', 'ecuSerialNumber', 'ecuSoftwareVersion',
                                         'eventDescription', 'Latitude', 'Longitude'])

In [102]:
#Concats the spn and fmi fields into a single field, then converts them back to an int

geo_codes['spnfmi'] = geo_codes.spn.astype('str').map(str) + geo_codes.fmi.astype('str')
geo_codes['spnfmi'] = geo_codes['spnfmi'].astype('int32')

#Converts the 'true' and 'false' values within [Value] so as to be consistent with the rest of the column's data

#geo_codes['Value'] = geo_codes['Value'].replace(['True', 'False'], [1, 0])

In [103]:
#Creates the columns we will need to input our time series/rolling data

#spn_list = geo_codes['spn'].unique().tolist()
#spn_list.sort()

#header_list = geo_codes.columns.tolist()
#new_header_list = header_list + spn_list

#geo_codes = geo_codes.reindex(columns = new_header_list)

In [146]:
geo_codes_spnfmi = pd.get_dummies(geo_codes, columns = ['spnfmi'])

In [147]:
geo_codes_spnfmi['EventTimeStamp'] = geo_codes_spnfmi['EventTimeStamp'].astype('datetime64[ns]')

In [148]:
geo_codes_spnfmi['EventTimeStamp'] = pd.to_datetime(geo_codes_spnfmi['EventTimeStamp'], utc = True)

In [149]:
#geo_codes_dropped = geo_codes_spnfmi.drop_duplicates('FaultId')

In [150]:
geo_codes_dropped = geo_codes_spnfmi.reset_index()

In [151]:
List = [x for x in geo_codes_dropped.columns if 'spnfmi_' in x]

In [153]:
geo_codes_dropped_rolling = geo_codes_dropped.groupby('EquipmentID').rolling(window="14d", on='EventTimeStamp')[List].sum().reset_index()#.drop(columns='EventTimeStamp')

In [154]:
geo_codes_dropped_rolling

,EquipmentID,EventTimeStamp,spnfmi_272,spnfmi_274,spnfmi_389,spnfmi_512,spnfmi_513,spnfmi_842,spnfmi_849,spnfmi_912,...,spnfmi_5209534,spnfmi_6553518,spnfmi_6553520,spnfmi_6553531,spnfmi_8812116,spnfmi_11486311,spnfmi_19660829,spnfmi_33504023,spnfmi_52103214,spnfmi_52428731
0,1327,2015-05-19 12:02:55+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1327,2015-05-26 08:11:45+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1327,2015-05-26 08:44:10+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1330,2015-04-15 20:15:00+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1330,2015-04-15 20:22:10+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200149,310,2018-06-28 21:51:13+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200150,310,2018-06-28 22:03:41+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200151,R1762,2015-02-24 13:45:06+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200152,R1762,2015-02-24 15:31:17+00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [169]:
merge_geo_codes = geo_codes_dropped.merge(geo_codes_dropped_rolling, left_index = True, right_index = True)

In [170]:
merge_geo_codes = merge_geo_codes.rename(columns = {'EquipmentID_x' : 'EquipmentID', 'EventTimeStamp_x' : 'EventTimeStamp'})

In [171]:
merge_geo_codes = merge_geo_codes.loc[:,~merge_geo_codes.columns.str.contains('_x', case=False)]

In [172]:
merge_geo_codes = merge_geo_codes.drop(columns=['index'])

In [173]:
merge_geo_codes

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,activeTransitionCount,ecuModel,ecuSerialNumber,ecuSoftwareVersion,eventDescription,...,spnfmi_5209534_y,spnfmi_6553518_y,spnfmi_6553520_y,spnfmi_6553531_y,spnfmi_8812116_y,spnfmi_11486311_y,spnfmi_19660829_y,spnfmi_33504023_y,spnfmi_52103214_y,spnfmi_52428731_y
0,4246,R1762,2015-02-24 13:45:06+00:00,1761,17,43,6X1u13D1500000000,79751302,05317106*04119044*051914190353*09400015*G1*BDR*,Low (Severity Low) Catalyst Tank Level,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4428,R1762,2015-02-24 15:31:17+00:00,1761,18,11,6X1u13D1500000000,79751302,05317106*04119044*051914190353*09400015*G1*BDR*,Low (Severity Medium) Catalyst Tank Level,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6439,R1762,2015-02-26 13:12:11+00:00,5848,9,1,6X1u13D1500000000,79751302,05317106*04119044*051914190353*09400015*G1*BDR*,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49403,310,2015-05-11 12:52:51+00:00,1569,31,126,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,Condition Exists Engine Protection Torque Derate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,117951,310,2015-07-10 17:23:05+00:00,1569,31,126,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,Condition Exists Engine Protection Torque Derate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200149,114802,1330,2015-07-08 17:18:14+00:00,3242,15,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,High (Severity Low) Aftertreatment 1 Particula...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200150,116052,1330,2015-07-09 14:12:48+00:00,3242,15,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,High (Severity Low) Aftertreatment 1 Particula...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200151,58809,1327,2015-05-19 12:02:55+00:00,3216,9,1,6X1u10D1500000000,79419774,04993120*00001782*082113134117*07700053*I0*BBZ*,Abnormal Update Rate Aftertreatment 1 Intake NOx,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200152,65879,1327,2015-05-26 08:11:45+00:00,3216,9,1,6X1u10D1500000000,79419774,04993120*00001782*082113134117*07700053*I0*BBZ*,Abnormal Update Rate Aftertreatment 1 Intake NOx,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [137]:
#onboard['Value'] = onboard['Value'].replace(['True', 'False'], [1, 0])

In [138]:
#onboard['Value'] = onboard['Value'].str.replace(',', '.')

In [139]:
#onboard['Value'] = pd.to_numeric(onboard['Value'])

In [140]:
onboard

,Id,Name,Value,FaultId
0,1,IgnStatus,NaN,1
1,2,EngineOilPressure,0.00000,1
2,3,EngineOilTemperature,96.74375,1
3,4,TurboBoostPressure,0.00000,1
4,5,EngineLoad,11.00000,1
...,...,...,...,...
12821621,12864020,EngineCoolantTemperature,181.40000,1248457
12821622,12864021,ParkingBrake,NaN,1248457
12821623,12864022,SwitchedBatteryVoltage,14.10000,1248457
12821624,12864023,DistanceLtd,28606.65625,1248457


In [141]:
#onboard_pivot = onboard.pivot_table(index = 'FaultId',
                                             columns = 'Name',
                                             values = 'Value').reset_index()

In [142]:
#onboard_pivot

Name,FaultId,AcceleratorPedal,BarometricPressure,CruiseControlSetSpeed,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,FuelLtd,FuelRate,FuelTemperature,IntakeManifoldTemperature,LampStatus,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,0.0,14.2100,66.48672,423178.70000,100.4,11.0,0.00,96.74375,0.00,...,12300.907429,0.000000,NaN,78.8,1023.0,NaN,0.00000,3276.75,NaN,0.00
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,16639.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187330,1248454,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1023.0,NaN,NaN,NaN,NaN,NaN
1187331,1248455,100.0,14.5000,64.62260,423937.90000,185.0,51.0,37.12,211.49370,1310.25,...,58979.184416,7.647805,32.0,98.6,18431.0,NaN,65.01096,NaN,73.2,7.83
1187332,1248456,0.0,14.3550,66.48672,465925.40000,186.8,62.0,41.18,212.84380,1340.75,...,65080.105870,8.995086,NaN,91.4,17407.0,NaN,66.57410,NaN,100.0,6.96
1187333,1248457,1.6,14.4275,67.72946,28606.65625,181.4,0.0,27.26,221.73120,863.25,...,4042.492826,0.000000,NaN,100.4,1023.0,NaN,11.84489,14.10,100.0,1.74


In [ ]:
#onboard_merge = onboard.merge(merge_geo_codes, how = 'inner', left_on = 'FaultId', right_on = 'RecordID')

Function (df) sort event time stamp, split off X%, split off bottom 20%

Apply it to each truck individually, via groupby, a bunch of dataframes, and then we reattach 

In [178]:
def train_test_brake (df):
    df = df.sort_values('EventTimeStamp')
    t_len = df.shape[0]
    t_perc80 = int(t_len * 0.8)
    df80 = df.iloc[0 : t_perc80, :]
    df20 = df.iloc[t_perc80 : , :]
    return(df80, df20) 

In [187]:
df = merge_geo_codes[merge_geo_codes['EquipmentID'].isin(['310', '1330'])]

In [188]:
df.shape

(78, 411)

In [182]:
df80, df20 = train_test_brake(df)

In [189]:
df80, df20 = df.groupby('EquipmentID').apply(train_test_brake)

In [203]:
brake_temp = df.groupby('EquipmentID').apply(train_test_brake)

In [212]:
df80 = pd.concat([x[0] for x in brake_temp])

In [215]:
df20 = pd.concat([x[1] for x in brake_temp])

In [216]:
df20

,RecordID,EquipmentID,EventTimeStamp,spn,fmi,activeTransitionCount,ecuModel,ecuSerialNumber,ecuSoftwareVersion,eventDescription,...,spnfmi_5209534_y,spnfmi_6553518_y,spnfmi_6553520_y,spnfmi_6553531_y,spnfmi_8812116_y,spnfmi_11486311_y,spnfmi_19660829_y,spnfmi_33504023_y,spnfmi_52103214_y,spnfmi_52428731_y
200147,69236,1330,2015-05-28 17:44:46+00:00,3251,0,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,High (Severity High) Aftertreatment 1 Particul...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200148,78610,1330,2015-06-05 19:24:04+00:00,3556,2,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,Incorrect Data Aftertreatment Fuel Injector 1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200149,114802,1330,2015-07-08 17:18:14+00:00,3242,15,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,High (Severity Low) Aftertreatment 1 Particula...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200150,116052,1330,2015-07-09 14:12:48+00:00,3242,15,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,High (Severity Low) Aftertreatment 1 Particula...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,549538,310,2016-08-03 22:33:27+00:00,97,15,70,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,High (Severity Low) Water In Fuel Indicator,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
52,550600,310,2016-08-04 21:14:56+00:00,97,16,1,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,High (Severity Medium) Water In Fuel Indicator,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53,676413,310,2017-01-02 20:03:09+00:00,5848,9,1,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
54,876806,310,2017-09-22 02:49:53+00:00,1569,31,126,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,Condition Exists Engine Protection Torque Derate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
55,1004216,310,2018-04-26 20:17:04+00:00,5394,5,1,6X1u13D1500000000,79756075,05317106*04133655*061416163421*09401361*G1*BDR*,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
56,1016600,310,2018-05-22 17:27:54+00:00,5394,7,1,6X1u13D1500000000,79756075,05317106*04133655*061416163421*09401361*G1*BDR*,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#geo_codes['Value'] = geo_codes['Value'].replace(['True', 'False'], [1, 0])

In [ ]:
#real_codes = onboard.merge(j1939, how = 'left', left_on = 'FaultId', right_on = 'RecordID')

In [ ]:
#geo_codes_dropped = geo_codes_spnfmi.drop_duplicates('FaultId')

In [92]:
merged_list = merge_geo_codes.loc[:, ~merge_geo_codes.columns.isin(['Name','Value'])].columns.tolist()

In [43]:
merge_geo_codes['Value'] = pd.to_numeric(merge_geo_codes['Value'])

In [47]:
merge_geo_codes

,FaultId,spn,fmi,Name,Value,activeTransitionCount,ecuModel,ecuSerialNumber,ecuSoftwareVersion,eventDescription,...,spnfmi_5209534_y,spnfmi_6553518_y,spnfmi_6553520_y,spnfmi_6553531_y,spnfmi_8812116_y,spnfmi_11486311_y,spnfmi_19660829_y,spnfmi_33504023_y,spnfmi_52103214_y,spnfmi_52428731_y
0,4246,1761,17,Throttle,47.2,43,6X1u13D1500000000,79751302,05317106*04119044*051914190353*09400015*G1*BDR*,Low (Severity Low) Catalyst Tank Level,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4428,1761,18,Throttle,67.2,11,6X1u13D1500000000,79751302,05317106*04119044*051914190353*09400015*G1*BDR*,Low (Severity Medium) Catalyst Tank Level,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,6439,5848,9,Throttle,0.0,1,6X1u13D1500000000,79751302,05317106*04119044*051914190353*09400015*G1*BDR*,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,49403,1569,31,IgnStatus,0.0,126,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,Condition Exists Engine Protection Torque Derate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,117951,1569,31,Throttle,0.0,126,6X1u13D1500000000,79756075,05317106*04133655*051914190353*09400015*G1*BDR*,Condition Exists Engine Protection Torque Derate,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200149,114802,3242,15,IgnStatus,1.0,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,High (Severity Low) Aftertreatment 1 Particula...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200150,116052,3242,15,IgnStatus,1.0,1,6X1u10D1500000000,79419775,04993120*00001524*040213150018*07700044*I0*BBZ*,High (Severity Low) Aftertreatment 1 Particula...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200151,58809,3216,9,IgnStatus,1.0,1,6X1u10D1500000000,79419774,04993120*00001782*082113134117*07700053*I0*BBZ*,Abnormal Update Rate Aftertreatment 1 Intake NOx,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
200152,65879,3216,9,IgnStatus,1.0,1,6X1u10D1500000000,79419774,04993120*00001782*082113134117*07700053*I0*BBZ*,Abnormal Update Rate Aftertreatment 1 Intake NOx,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
merge_geo_codes['Name'].unique()

array(['Throttle', 'IgnStatus', 'FuelLevel', 'BarometricPressure',
       'AcceleratorPedal', 'Speed'], dtype=object)

In [45]:
merge_geo_codes_pivot = merge_geo_codes.pivot_table(index = merged_list,
                                             columns = 'Name',
                                             values = 'Value').reset_index()

In [46]:
merge_geo_codes_pivot

Name,FaultId,spn,fmi,activeTransitionCount,ecuModel,ecuSerialNumber,ecuSoftwareVersion,eventDescription,Latitude,Longitude,...,spnfmi_19660829_y,spnfmi_33504023_y,spnfmi_52103214_y,spnfmi_52428731_y,AcceleratorPedal,BarometricPressure,FuelLevel,IgnStatus,Speed,Throttle
0,6,111,17,1,6X1u10D1500000000,79466580,04993120*00025921*082113134117*07700053*I0*BBZ*,Low (Severity Low) Engine Coolant Level,33.043564,-96.179722,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN
1,8,111,17,1,6X1u10D1500000000,79463845,04993120*00022630*082113134117*07700053*I0*BBZ*,Low (Severity Low) Engine Coolant Level,38.228796,-84.582500,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN
2,9,97,15,1,6X1u10D1500000000,79463845,04993120*00022630*082113134117*07700053*I0*BBZ*,High (Severity Low) Water In Fuel Indicator,38.228796,-84.582500,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN
3,11,111,17,1,6X1u10D1500000000,79618845,04993120*00184697*060712145341*07700027*I0*BBZ*,Low (Severity Low) Engine Coolant Level,40.158055,-80.151018,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN
4,17,111,17,1,6X1u10D1500000000,79623056,04993120*00190002*082113134117*07700053*I0*BBZ*,Low (Severity Low) Engine Coolant Level,38.279629,-78.935509,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193057,1248444,97,15,63,6X1u13D1500000000,79903052,04358814*06027020*051718174436*09401683*G1*BDR*,High (Severity Low) Water In Fuel Indicator,39.654074,-79.437592,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,50.0
193058,1248446,97,15,90,6X1u13D1500000000,79902398,04358814*06027888*012618213404*09401682*G1*BDR*,High (Severity Low) Water In Fuel Indicator,37.826157,-79.442870,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,100.0
193059,1248452,111,18,93,6X1u13D1500000000,79904453,04358814*06030918*051718174436*09401683*G1*BDR*,Low (Severity Medium) Engine Coolant Level,39.015694,-77.031157,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,100.0
193060,1248455,1569,31,5,6X1u13D1500000000,79932020,04358814*06099720*030816202706*09400153*G1*BDR*,Condition Exists Engine Protection Torque Derate,34.390740,-79.461805,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,73.2


## Initial Rolling Count Code Testing 

In [ ]:
List=Truck310.loc[:,'spnfmi_1001':'spnfmi_974'].columns.to_list()

In [ ]:
Truck310 = geo_codes_spnfmi[geo_codes_spnfmi['EquipmentID']=="310"].drop_duplicates('FaultId')

In [ ]:
Truck310

In [ ]:
#Truck310.loc[:,['rolling_spnfmi_156931','rolling_spnfmi_64131']] = 
Truck310rolling = Truck310.rolling(window="14d", on='EventTimeStamp')[List].sum().drop(columns='EventTimeStamp')

In [ ]:
Truck310merged = Truck310.merge(Truck310rolling, left_index=True,right_index=True)

In [ ]:
Truck310merged[['spnfmi_156931_x','spnfmi_156931_y','spnfmi_64131_x','spnfmi_64131_y','EventTimeStamp']].head(20)

In [ ]:
#truck310.loc[:,['rolling_156931', 'rolling_64131']] = truck310.rolling(window = '14d', on = 'EventTimeStamp')['spnfmi_156931', 'spnfmi_64131'].sum().drop(columns = 'EventTimeStamp')

#truck310_rolling = truck310.rolling(window = '14d', on = 'EventTimeStamp')[List].sum().drop(columns = 'EventTimeStamp')

In [ ]:
#truck310_rolling.sort_values('spnfmi_9715', ascending = True)